In [1]:
import pandas as pd 
import numpy as np 

In [2]:
subject_id = 'S01'
game_id = 'G3'
label = 0
data_path = '../database'

data = pd.read_csv(data_path + '/' + subject_id + game_id + 'AllChannels.csv')


In [3]:
data.head()

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,Unnamed: 14
0,52.5434,-33.7436,91.1077,-19.28220,-57.8463,48.2051,1.9284,-11.0870,42.4205,-1.92840,74.2359,-6.2667,22.1742,-4.8205,NaN
1,48.9088,-29.3086,82.2668,-12.82260,-62.0883,37.1181,2.7766,-10.4218,30.2343,-6.15100,65.4434,-8.7828,8.3107,-2.6031,NaN
2,43.0825,-24.6576,79.2592,-4.82220,-55.4704,29.1066,-0.2823,-9.3145,34.6870,0.48477,62.9629,-11.6303,5.8837,-5.3392,NaN
3,43.3896,-21.7323,79.8063,-0.19451,-39.6090,35.5549,1.6628,-8.7559,42.7287,-1.47250,57.7388,-14.7891,11.7972,-4.5369,NaN
4,46.5708,-23.8029,75.4999,-5.48570,-37.2327,28.1190,-2.2933,-4.8561,31.4878,2.47220,60.5410,-9.5632,7.7149,-3.3003,NaN


In [4]:
# Grab data 6 channles 
data_AF3 = data['AF3']
data_AF4 = data['AF4']
data_F3 = data['F3']
data_F4 = data['F4']
data_F7 = data['F7']
data_F8 = data['F8']
data_6_channels = np.vstack((data_AF3, data_AF4, data_F3, data_F4, data_F7, data_F8))
print(data_6_channels.shape)


(6, 38252)


In [6]:
# Grab filter 
filter_path = './filter_coefficients.csv'
filters = pd.read_csv(filter_path)

filter_band_1 = filters['Filter 1']
filter_band_2 = filters['Filter 2']
filter_band_3 = filters['Filter 3']
filter_band_4 = filters['Filter 4']


In [7]:
from scipy import signal

Beta_F7 = signal.filtfilt(filter_band_4, 1, data_F7)
Beta_F8 = signal.filtfilt(filter_band_4, 1, data_F8)
diff_beta = Beta_F7 - Beta_F8 


label = np.full(len(diff_beta), 1, dtype=int)
data_with_label = np.vstack((label, diff_beta))

print(data_with_label)
# np.save('datawithlabel-G3', data_with_label)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[[   1.            1.            1.         ...    1.
     1.            1.        ]
 [-132.36335104 -129.59064773 -126.81961648 ...  -10.79212337
   -11.01035118  -11.22870725]]


In [8]:
from scipy import signal
import numpy as np

NUM_BANDS = 4
SAMPLING_FREQ = 128
WINDOW_LENGTH = 4
DATA_CHN = 6
SHIFTED_SEC = 2 
NUM_HJORTH_PARAM = 3 * 4 # 3 Param in each bands 

# Filter Bank Convolution  (((data_chn*num_bands)+1, data_len))
data_filtered_array = np.zeros(((DATA_CHN*NUM_BANDS)+1, data_6_channels.shape[1]))
for i in range(DATA_CHN):
    data_filtered_array[i*NUM_BANDS+0][:]  = signal.filtfilt(filter_band_1, 1, data_6_channels[i])
    data_filtered_array[i*NUM_BANDS+1][:]  = signal.filtfilt(filter_band_2, 1, data_6_channels[i])      
    data_filtered_array[i*NUM_BANDS+2][:]  = signal.filtfilt(filter_band_3, 1, data_6_channels[i])
    data_filtered_array[i*NUM_BANDS+3][:]  = signal.filtfilt(filter_band_4, 1, data_6_channels[i])

print(data_filtered_array.shape)


(25, 38252)


In [9]:
# Add event channel
data_filtered_array[-1, :] = label
print(f"Shape after Convolve is {data_filtered_array.shape}")
print(f'Count 2: {np.count_nonzero(data_filtered_array[-1]==1)}')


Shape after Convolve is (25, 38252)
Count 2: 38252


In [16]:
import numpy as np
from scipy.signal import hanning
from scipy.stats import moment

# Assuming 'data' is your input data with shape (24, 30000)
# You need to define the functions to calculate Energy and Hjorth parameters
def calculate_energy(signal):
    return np.sum(signal ** 2)

def hjorth_first_parameter(signal):
    return np.mean(np.diff(signal)) ** 2

def hjorth_second_parameter(signal):
    return np.var(np.diff(signal)) / np.var(signal)

def hjorth_third_parameter(signal):
    diff2 = np.diff(signal, n=2)
    return np.var(diff2) / np.var(signal)

# Parameters for the windowing and overlapping
sampling_rate = 128
seconds = 4
window_size = sampling_rate * seconds
overlap = int(0.5*window_size)

# Initialize an empty array to store the results
result_rows = []

# Iterate through each row of the data
for row in data_filtered_array:
    row_results = []
    
    # Apply the functions to each window with overlap
    for i in range(0, len(row) - window_size + 1, overlap):
        window = row[i:i + window_size]
        
        energy = calculate_energy(window)
        hjorth_1 = hjorth_first_parameter(window)
        hjorth_2 = hjorth_second_parameter(window)
        hjorth_3 = hjorth_third_parameter(window)
        
        row_results.append([energy, hjorth_1, hjorth_2, hjorth_3])
    
    result_rows.extend(row_results)

# Convert the results to a NumPy array for easier manipulation
result_array = np.array(result_rows)

# Transpose the result_array to achieve the desired shape (96, xxxx)
result_array = result_array.T

# Now result_array has the shape (96, xxxx)
print(result_array.shape)
# The transposition step result_array = result_array.T will convert the shape from (xxxx, 96) to (96, xxxx) as you requested.






(4, 3700)


/var/folders/fx/s9d23h851bqgcdp14ny244k80000gn/T/ipykernel_20460/4245989875.py:14: RuntimeWarning: invalid value encountered in scalar divide
  return np.var(np.diff(signal)) / np.var(signal)
/var/folders/fx/s9d23h851bqgcdp14ny244k80000gn/T/ipykernel_20460/4245989875.py:18: RuntimeWarning: invalid value encountered in scalar divide
  return np.var(diff2) / np.var(signal)


In [ ]:
# Calculate Energy perband
print(f"--------Calculating Energy {subject_id}{game_id}--------")

feature_array = np.zeros((((NUM_BANDS+NUM_HJORTH_NUM)*DATA_CHN)+1, int(data_filtered_array.shape[1]/(SAMPLING_FREQ*SHIFTED_SEC))))

for band in range((NUM_BANDS+NUM_HJORTH_PARAM) * DATA_CHN):
    start = 0 
    end = SAMPLING_FREQ * WINDOW_LENGTH
    count = 0
    overlapping = int(0.5 * end)

    if (feat != (NUM_BANDS+NUM_HJORTH_NUM) * DATA_CHN):  
        while (end <= data_filtered_array.shape[1]):
            new_sub_array = np.zeros((1, int(end)))
            new_sub_array = data_filtered_array[feat][:][start: end]
            energy_one_band = np.sum(np.power(new_sub_array, 2))
            feature_array[feat][count] = energy_one_band
            start += overlapping
            end += overlapping
            count += 1
            print(f'Feat {feat} done at chn {count}!')
    
feature_array[-1, :] = label

print(feature_array.shape)
filename = subject_id + "-" + game_id + "-with-labels.npy"
np.save(filename, feature_array)
print(f'Count 1: {np.count_nonzero(feature_array[-1]==1)}')
